In [97]:
# importing the required packages
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [98]:
# reading the input file
data = pd.read_csv(r"C:\Users\Chandu Jagarlamundi\Desktop\Thesis_Wind data\Data Wind_extern\data_eng.csv")

In [99]:
# deleting the columns which have lots of missing values
columns_data = data.columns
for i in columns_data:
    if data[i].isna().sum()>(len(data)/2):
        print("The number of missing values in the column {} are {} ".format(i,data[i].isna().sum()))
        data.drop([i], axis = 1, inplace = True)
columns_data = data.columns

The number of missing values in the column apparent power are 780813 
The number of missing values in the column generator speed are 437865 
The number of missing values in the column nacelle view are 780813 
The number of missing values in the column Digital 1 are 780813 
The number of missing values in the column Digital 2 are 780813 
The number of missing values in the column Three-phase current controller Setpoint are 780813 
The number of missing values in the column wind direction deviation are 780813 
The number of missing values in the column Average power 5 sec are 780813 
The number of missing values in the column Average power 30 sec are 780813 
The number of missing values in the column switched-on reactive power are 780813 
The number of missing values in the column performance class are 780813 
The number of missing values in the column Condition Sheet are 780813 
The number of missing values in the column Number of comp. levels are 780813 


In [100]:
# deleting the columns which have single values
for i in columns_data:
    if len(data[i].value_counts())<2:
        print("{} is eliminated".format(i))
        data.drop([i], axis = 1, inplace = True)
columns_data = data.columns

Time difference is eliminated
fast rateplay is eliminated


In [101]:
data.Equipment = data.Equipment[data.Equipment!='Anlage']
# dropping the missing rows
data.dropna(axis = 0, inplace=True)

In [102]:
data.Equipment = pd.Categorical(data.Equipment, categories=data.Equipment.unique()).codes

In [103]:
# the no.of different values in system status column
len_status = len(data.loc[:, "system status"].value_counts())

In [104]:
data.drop(['Date(Remote)', 'Time(Remote)', 'Date(Server)', 'Time(Server)', "operating state"], axis=1, inplace=True)

In [105]:
# defining the target and removing it from the data
target = data["system status"]
data.drop(["system status"], axis=1, inplace=True)
columns_data = len(data.columns)

In [106]:
# reading the anlagestatus file, to get the details regarding anlage status
status_data = pd.read_excel(r"C:\Users\Chandu Jagarlamundi\Desktop\Thesis_Wind data\Data Wind_extern\system_status.xlsx")

In [107]:
status_data.columns

Index(['Status_Number', 'status_text', 'Stop_with_description', 'Reset',
       'error_description', 'availability_key', 'Reset.1'],
      dtype='object')

In [108]:
# getting the true values of systemstatus
common_status = np.intersect1d(status_data['Status_Number'], target)

In [109]:
# values that contain decimals
float_status = target[~target.isin(common_status)]

In [110]:
target_valid = target[~target.isin(float_status)]

In [123]:
status = pd.DataFrame()
status["Status_number"] = status_data.Status_Number
status["status_text"] = status_data.status_text
status.dropna(axis=0, inplace=True)

In [124]:
status_text = status.set_index("Status_number").T.to_dict('list')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [126]:
target_valid = target_valid.map(status_text)

In [130]:
target_valid.value_counts()

[NO ERRORS.]                                           541595
[STORM SHUTDOWN]                                        83514
[PROGRAM START SPS]                                     36135
[MANUAL STOP]                                           24687
[PITCH AXIS 2 SPEED LOSS DETECTED]                       1118
[INVALID PWA, ROTOR BRIDGE PLATE]                         652
[PITCH AXIS 3 SPEED LOSS DETECTED]                        418
[EXCESS FREQUENCY]                                        220
[MISSING MARKER PULSES PITCH AXIS 2 DETECTED]             197
[SYSTEM OK]                                               157
[PITCH BEARING ROTARY FUNCTION RUNS]                      139
[PITCH AXIS 3 PHASE FAILURE DETECTED]                     117
[UNDERVOLTAGE]                                            105
[PITCH AXIS 3 SAFETY RELAYS FAILED]                       100
[PITCH AXIS 1 SPS COMMUNICATION NOT OK]                    99
[MIS-CONFIGURATION BY PARAMETER]                           94
[PITCH C